In [117]:
MIN_PRIME = 100
MAX_PRIME = 1050

msg = 12345678900987654321

In [118]:
import sympy
import random

def genPrime():
    return sympy.randprime(MIN_PRIME, MAX_PRIME)

def fermatPrime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def mmi(E, phiN):
    return pow(E, -1, phiN)

def calcN(P, Q):
    return P*Q

def calcPhiN(P, Q):
    return (P-1) * (Q-1)

def toBinary(dec):
    return "{0:b}".format(dec)

def toDecimal(bin):
    return int(bin, 2)

def concatenateInBinary(prev, tail):
    prev = toBinary(prev)
    tail = toBinary(tail)

    while len(prev) < len(tail):
        prev = "0" + prev

    while len(tail) < len(prev):
        tail = "0" + tail

    return toDecimal(prev + tail)

def splitInBinary(bin):
    bin = toBinary(bin)
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(toDecimal(prev), toDecimal(tail))



# Voraussetzungen

In [119]:
a_P = genPrime()
a_Q= genPrime()


a_PhiN = calcPhiN(a_P, a_Q)
a_N = calcN(a_P, a_Q)

a_E = fermatPrime()
a_D = mmi(a_E, a_PhiN)

Attacker = {
    "P" : a_P,
    "Q" : a_Q,
    "PhiN" : a_PhiN,
    "N" : a_N,
    "E" : a_E,
    "D" : a_D
}

print(Attacker)

{'P': 907, 'Q': 353, 'PhiN': 318912, 'N': 320171, 'E': 65537, 'D': 280961}


# Generierung und Verschlüsselung

## Schritt 1

In [120]:
P = genPrime()

vP = pow(P, a_E ,a_N)

Attacker["vP"] = vP

print(P, vP)

877 39825


## Schritt 2 und 3

In [129]:
Q = 4
R = 1

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
while(not sympy.isprime(Q) and not R == 0):
    t = genPrime()

    tempN = concatenateInBinary(vP, t)

    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    R = tempN % P
    Q = int((tempN - R) / P)

    Attacker["R"] = R

print(R, Q)

print(vP, t)
print(toBinary(vP), toBinary(t))
print(tempN)
print(toBinary(tempN))

20 2976023
39825 991
1001101110010001 1111011111
2609972191
10011011100100010000001111011111


## Schritt 4

In [130]:
N = calcN(P, Q)

print(N)

2609972171


## Schritt 5

In [131]:
PhiN = calcPhiN(P, Q)
E = fermatPrime()

print(P, Q)
print(E)

D = mmi(E, PhiN)

print(PhiN, E, D)

877 2976023
17
2606995272 17 2453642609


In [132]:
User = {
    "P" : P,
    "Q" : Q,
    "PhiN" : PhiN,
    "N" : N,
    "E" : E,
    "D" : D
}

pub_User = {
    "P" : P,
    "Q" : Q,
    "PhiN" : PhiN,
    "N" : N,
    "E" : E,
    "D" : D
}

print(User)

{'P': 877, 'Q': 2976023, 'PhiN': 2606995272, 'N': 2609972171, 'E': 17, 'D': 2453642609}


## Schritt 6

In [133]:
cipher = pow(msg, User["E"], User["N"])
signature = pow(msg, User["D"], User["N"])

# Angriff

## Schritt 1

In [134]:
# Read the public key of user

## Schritt 2

In [135]:
exp_vP = splitInBinary(User["N"])[0]

print(toBinary(User["N"]))
print(toBinary(Attacker["vP"]))
print(toBinary(exp_vP))

print(toBinary(Attacker["R"]))
print(Attacker["R"])




10011011100100010000001111001011
1001101110010001
1001101110010001
10100
20


## Schritt 3

In [136]:
exp_P = pow(exp_vP, Attacker["D"], Attacker["N"])

exp_alt_P = pow(exp_vP+1, Attacker["D"], Attacker["N"])

print(exp_P, exp_alt_P, P)

877 105143 877


123
{'P': 907, 'Q': 353, 'PhiN': 318912, 'N': 320171, 'E': 65537, 'D': 280961, 'vP': 39825, 'R': 20}
